In [1]:
import torch
import torchvision
import numpy as np
import gradient_reversal
import joblib
import os

In [2]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Import WIKI_EVE

In [3]:
import wiki_eve_max.model as models
import wiki_eve_max.loss as loss
import wiki_eve_max.transform as transform
import wiki_eve_max.dataset as dataset
import wiki_eve_max.work as work

In [4]:
CWD = os.getcwd()#"/mnt/fast_data"
META_FILE = os.path.join(CWD, "DATASETS_TEST0/NP/random_spacing/mean/32", "meta67.dump")
DATASET = dataset.BFIKEYSTROKE(
    meta_file=META_FILE,
    transform=torchvision.transforms.Compose([transform.Absolute()])
)

SAVE = True
TRAIN = True
MODEL_PATH_SAVE = "MODEL_WEIGHTS/random/MODEL_WEIGHTS_Adv_40k_samples_05_room_10epoch"

In [5]:
print(len(DATASET),len(DATASET.m_domains),DATASET.m_num_domain, DATASET.m_num_class)

140000 14000 14000 10


In [6]:
train_data, test_data = DATASET.split(train_p=0.9, total_samples=40000)

In [7]:
torch.cuda.empty_cache()

SC*NR*NC

In [8]:

learning_rate = 1e-3
batch_size = 10
epochs =1
channels = 234*4*2

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

model_path = "MODEL_WEIGHTS/random/MODEL_WEIGHTS_Adv_40k_samples_05_room_8epoch"
model = models.WIKI_EVE_1D(
    conv_in_channels = channels,
    conv_out_channels = channels,
    conv_kernel_size = 16,
    conv_stride = 1,
    GF_output_size = 150,
    GC_labels = 10,
    GD_labels = 2,#DATASET.m_num_domain,
    groups=channels,
    lambda_domain = -0.5
)

print(model)

if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, weights_only=True))

model.to(torch.device(device=device))

loss_fn = loss.AdvLernLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

WIKI_EVE_1D(
  (GF): Sequential(
    (0): Conv1d(1872, 1872, kernel_size=(16,), stride=(1,), groups=1872)
    (1): ReLU()
    (2): AdaptiveMaxPool1d(output_size=450)
    (3): Conv1d(1872, 1872, kernel_size=(16,), stride=(1,), groups=1872)
    (4): ReLU()
    (5): AdaptiveMaxPool1d(output_size=150)
  )
  (GC): Sequential(
    (0): Linear(in_features=280800, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=1000, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1000, out_features=10, bias=True)
  )
  (GD): Sequential(
    (0): Linear(in_features=280800, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=2, bias=True)
    (3): Sigmoid()
  )
)


In [9]:
if TRAIN:
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        work.train_loop(train_dataloader, model, loss_fn, optimizer, batch_size=batch_size, device=device, v=False, batch_no_print=100)
        work.test_loop(test_dataloader, model, loss_fn, device=device, v=False, batch_no_print=100)
    print("Done!")

Epoch 1
-------------------------------
loss: -0.006365  [   10/36000]
loss: -0.181136  [ 1010/36000]
loss: -0.003720  [ 2010/36000]
loss: -0.187257  [ 3010/36000]
loss: -0.288775  [ 4010/36000]
loss: -0.336425  [ 5010/36000]
loss: -0.143084  [ 6010/36000]
loss: 0.223378  [ 7010/36000]
loss: 0.181910  [ 8010/36000]
loss: 0.249149  [ 9010/36000]
loss: -0.253564  [10010/36000]
loss: 0.017897  [11010/36000]
loss: -0.101030  [12010/36000]
loss: 0.334690  [13010/36000]
loss: -0.190764  [14010/36000]
loss: -0.107878  [15010/36000]
loss: -0.128022  [16010/36000]
loss: 0.563371  [17010/36000]
loss: -0.014854  [18010/36000]
loss: 0.339650  [19010/36000]
loss: -0.113668  [20010/36000]
loss: -0.117125  [21010/36000]
loss: -0.245499  [22010/36000]
loss: -0.143988  [23010/36000]
loss: 0.217723  [24010/36000]
loss: 0.334242  [25010/36000]
loss: 0.292122  [26010/36000]
loss: 0.574956  [27010/36000]
loss: -0.222148  [28010/36000]
loss: -0.197523  [29010/36000]
loss: 0.036723  [30010/36000]
loss: 0.050

In [10]:
if SAVE:
    torch.save(model.state_dict(), MODEL_PATH_SAVE)

In [11]:
work.test_loop(test_dataloader, model, loss_fn, device=device, v=False, batch_no_print=1)


* Test Error: 
 Accuracy (label): 90.0%, Accuracy (domain): 0.0%, Avg loss: 0.031969 


* Test Error: 
 Accuracy (label): 90.0%, Accuracy (domain): 0.0%, Avg loss: -0.030497 


* Test Error: 
 Accuracy (label): 92.5%, Accuracy (domain): 0.0%, Avg loss: -0.074508 


* Test Error: 
 Accuracy (label): 88.0%, Accuracy (domain): 0.0%, Avg loss: 0.091146 


* Test Error: 
 Accuracy (label): 86.7%, Accuracy (domain): 0.0%, Avg loss: 0.054331 


* Test Error: 
 Accuracy (label): 87.1%, Accuracy (domain): 0.0%, Avg loss: 0.032031 


* Test Error: 
 Accuracy (label): 86.2%, Accuracy (domain): 0.0%, Avg loss: 0.027553 


* Test Error: 
 Accuracy (label): 86.7%, Accuracy (domain): 0.0%, Avg loss: 0.015997 


* Test Error: 
 Accuracy (label): 87.0%, Accuracy (domain): 0.0%, Avg loss: 0.022144 


* Test Error: 
 Accuracy (label): 86.4%, Accuracy (domain): 0.0%, Avg loss: 0.003254 


* Test Error: 
 Accuracy (label): 86.7%, Accuracy (domain): 0.0%, Avg loss: -0.002953 


* Test Error: 
 Accuracy (la